<h1 style='text-align:center'>Titanic - Machine Learning from Disaster.</h1>
<br>

![](http://media.giphy.com/media/1Nk9bIidJVTy0/giphy.gif)

Objective: To predict whether a passenger survive or not, based on the other features

In [ ]:
### Importing Packages

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#import train and test data
train=pd.read_csv('data/train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
#### Percentage of Missing Values
np.round(train.isnull().sum()/len(train) * 100,0)

In [ ]:
# Remove the unused columns - We also remove cabin since it has more than 75% missing values
train.drop(['Ticket','Name', 'Embarked', 'SibSp', 'Parch', 'Cabin'],axis=1,inplace=True)
train.set_index('PassengerId', inplace=True)
train.head()

In [ ]:
train.info()

In [ ]:
### Survived Distribution
np.round(train.Survived.value_counts(normalize=True)*100,0)

In [ ]:
train.describe()

In [ ]:
train.Age.isna().sum()

In [ ]:
#### Store Age Median into a variable
age_median = train.Age.median()
age_median

In [ ]:
def age_missing_fill(X):
    if np.isnan(X):
        return age_median
    else:
        return X

In [ ]:
train['Age'] = train.Age.apply(age_missing_fill)

In [ ]:
pd.crosstab(train.Sex, train.Survived)

In [ ]:
train.boxplot(column=['Fare'])
plt.show()

#### Detecting Outliers

In [ ]:
q75, q25 = np.percentile(train.Fare, [75,25])
iqr = q75 - q25
upper_whisker = q75 + 1.5 * iqr
lower_whisker = q25 - 1.5 * iqr

In [ ]:
def check(x, ul, ll):
    if ul>=x>=ll:
        return x

In [ ]:
print("Percentage of Outliers in Fare:",len(train[train.Fare.apply(check, args = (upper_whisker, lower_whisker)).isnull()]['Fare'])/len(train) * 100)

# Quiz 5) How to Fill the outliers here??

In [ ]:
train.Fare.describe()

In [ ]:
train.groupby('Pclass')['Fare'].mean()

In [ ]:
def transform_fare(x):
    if x > upper_whisker:
        return upper_whisker
    return x

train['Fare'] = train['Fare'].apply(lambda x: transform_fare(x))
train['Fare'].plot.hist(bins=100, title='Frequency distribution of mean transformed Trip duration');

In [ ]:
train.Fare.describe()

#### Heat Map

In [ ]:
plt.subplots(figsize = (15,8))
sns.heatmap(train.corr(), annot=True,cmap="PiYG")
plt.title("Correlations Among Features", fontsize = 20);

In [ ]:
train = pd.get_dummies(train)

In [ ]:
train.head()

In [ ]:
X = train.iloc[:, 1:].values
y = train.iloc[:, 0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 45)

In [ ]:
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

In [ ]:
print("Accuracy of the model:", accuracy_score(y_test, y_pred))

In [ ]:
np.bincount(y_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))